In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# Carrengando um complemento para visualizar quanto tempo demoram as tarefas

%load_ext autotime

In [3]:
df_iptu = pd.read_csv('IPTU/IPTU_2020.zip', \
                      compression='zip', encoding='iso-8859-9', \
                      sep=';', decimal=',')

In [4]:
df_iptu['SQLC'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:10] + \
df_iptu['NUMERO DO CONDOMINIO'].str[0:2]

In [5]:
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'lote'] = \
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'NUMERO DO CONTRIBUINTE'].str[6:10]

In [6]:
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00', 'lote'] = '0000'

In [7]:
df_iptu['SQLC-lote'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:6] + \
df_iptu['lote'] + \
df_iptu['NUMERO DO CONDOMINIO'].str[0:2]

In [8]:
df_iptu[['SQLC','SQLC-lote']]

,SQLC,SQLC-lote
0,001003000100,001003000100
1,001003000200,001003000200
2,001003000300,001003000300
3,001003000400,001003000400
4,001003000500,001003000500
...,...,...
3498639,310118023801,310118000001
3498640,310118023901,310118000001
3498641,310118024001,310118000001
3498642,310119000100,310119000100


In [29]:
campos = ['SQLC',
          'SQLC-lote',
         'NOME DE LOGRADOURO DO IMOVEL', 
         'NUMERO DO IMOVEL',
         'COMPLEMENTO DO IMOVEL',
         'CEP DO IMOVEL',
         'TIPO DE USO DO IMOVEL']

In [30]:
# df_iptu['NUMERO DO IMOVEL'] = df_iptu['NUMERO DO IMOVEL'][df_iptu['NUMERO DO IMOVEL'].notna()].astype(int)
df_iptu.loc[df_iptu['NUMERO DO IMOVEL'].isna(), 'NUMERO DO IMOVEL'] = 0
df_iptu = df_iptu.astype({'NUMERO DO IMOVEL':'int32'})
# df_iptu['numero_de_porta'] = df_iptu['NUMERO DO IMOVEL'][df_iptu['NUMERO DO IMOVEL'].notna()].astype('int32')

In [31]:
df_iptu['NOME DE LOGRADOURO DO IMOVEL'] = df_iptu['NOME DE LOGRADOURO DO IMOVEL'].str.title()

In [32]:
df_iptu['COMPLEMENTO DO IMOVEL'] = df_iptu['COMPLEMENTO DO IMOVEL'].str.title()

In [33]:
df_coords = pd.read_csv('resultados/lotes-por-pontos.csv', dtype={'SQLC-lote': object})

In [34]:
df_coords['X'] = df_coords.x
df_coords['Y'] = df_coords.y

In [35]:
df_coords.dtypes

SQLC-lote     object
x            float64
y            float64
X            float64
Y            float64
dtype: object

In [36]:
# df_iptu.dtypes

In [37]:
df_enderecos = df_iptu[campos].merge(df_coords, on='SQLC-lote', how='left')

In [38]:
df_enderecos

,SQLC,SQLC-lote,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,CEP DO IMOVEL,TIPO DE USO DO IMOVEL,x,y,X,Y
0,001003000100,001003000100,R S Caetano,13,NaN,01104-001,Loja,-46.632863,-23.534357,-46.632863,-23.534357
1,001003000200,001003000200,R S Caetano,19,NaN,01104-001,Loja,-46.632790,-23.534376,-46.632790,-23.534376
2,001003000300,001003000300,R S Caetano,27,NaN,01104-001,Loja,-46.632722,-23.534379,-46.632722,-23.534379
3,001003000400,001003000400,R S Caetano,33,NaN,01104-001,Loja,-46.632657,-23.534385,-46.632657,-23.534385
4,001003000500,001003000500,R S Caetano,39,NaN,01104-001,Loja,-46.632585,-23.534343,-46.632585,-23.534343
...,...,...,...,...,...,...,...,...,...,...,...
3503968,310118023801,310118000001,Av Eng Armando De Arruda Pereira,1827,Ap 42,04309-011,Apartamento em condomínio,-46.640787,-23.642935,-46.640787,-23.642935
3503969,310118023901,310118000001,Av Eng Armando De Arruda Pereira,1827,Ap 43,04309-011,Apartamento em condomínio,-46.640787,-23.642935,-46.640787,-23.642935
3503970,310118024001,310118000001,Av Eng Armando De Arruda Pereira,1827,Ap 44,04309-011,Apartamento em condomínio,-46.640787,-23.642935,-46.640787,-23.642935
3503971,310119000100,310119000100,R Cotipora,0,NaN,04317-060,Terreno,-46.631329,-23.644451,-46.631329,-23.644451


In [39]:
df_enderecos[df_enderecos.X.isna()]

,SQLC,SQLC-lote,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,CEP DO IMOVEL,TIPO DE USO DO IMOVEL,x,y,X,Y
1539,001013003400,001013003400,R Possidonio Inacio,34,38 E 44,01103-030,"Prédio de apartamento, não em condomínio, de u...",NaN,NaN,NaN,NaN
11661,001049067100,001049067100,R Florencio De Abreu,333,337,01029-000,Loja,NaN,NaN,NaN,NaN
11662,001049067200,001049067200,R Florencio De Abreu,343,NaN,01029-000,Loja,NaN,NaN,NaN,NaN
11663,001049067300,001049067300,R Florencio De Abreu,353,355,01029-000,Loja,NaN,NaN,NaN,NaN
11664,001049067400,001049067400,R Carlos De Sousa Nazare,54,NaN,01025-000,Loja,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3503303,310110003900,310110003900,R S Borja,98,10 A Qd 3,04320-060,Residência,NaN,NaN,NaN,NaN
3503304,310110004000,310110004000,R Jose De Alcantara,91,NaN,04324-000,Residência,NaN,NaN,NaN,NaN
3503305,310110004100,310110004100,R S Borja,158,NaN,04320-060,Residência,NaN,NaN,NaN,NaN
3503306,310110004200,310110004200,R Jose De Alcantara,159,NaN,04324-000,Residência,NaN,NaN,NaN,NaN


In [40]:
campos_exportacao = \
        ['SQLC',
         'X', 'Y', 
         'NOME DE LOGRADOURO DO IMOVEL', 
         'NUMERO DO IMOVEL',
         'COMPLEMENTO DO IMOVEL',
         'CEP DO IMOVEL',
         'TIPO DE USO DO IMOVEL']

In [41]:
campos_renomear = \
        {'SQLC': 'name',
         'X': 'lon', 'Y': 'lat', 
         'NOME DE LOGRADOURO DO IMOVEL': 'street', 
         'NUMERO DO IMOVEL': 'number',
         'COMPLEMENTO DO IMOVEL': 'housenumber',
         'CEP DO IMOVEL': 'postcode'}

In [42]:
df_enderecos.rename(columns=campos_renomear, inplace=True)

In [43]:
df_enderecos

,name,SQLC-lote,street,number,housenumber,postcode,TIPO DE USO DO IMOVEL,x,y,lon,lat
0,001003000100,001003000100,R S Caetano,13,NaN,01104-001,Loja,-46.632863,-23.534357,-46.632863,-23.534357
1,001003000200,001003000200,R S Caetano,19,NaN,01104-001,Loja,-46.632790,-23.534376,-46.632790,-23.534376
2,001003000300,001003000300,R S Caetano,27,NaN,01104-001,Loja,-46.632722,-23.534379,-46.632722,-23.534379
3,001003000400,001003000400,R S Caetano,33,NaN,01104-001,Loja,-46.632657,-23.534385,-46.632657,-23.534385
4,001003000500,001003000500,R S Caetano,39,NaN,01104-001,Loja,-46.632585,-23.534343,-46.632585,-23.534343
...,...,...,...,...,...,...,...,...,...,...,...
3503968,310118023801,310118000001,Av Eng Armando De Arruda Pereira,1827,Ap 42,04309-011,Apartamento em condomínio,-46.640787,-23.642935,-46.640787,-23.642935
3503969,310118023901,310118000001,Av Eng Armando De Arruda Pereira,1827,Ap 43,04309-011,Apartamento em condomínio,-46.640787,-23.642935,-46.640787,-23.642935
3503970,310118024001,310118000001,Av Eng Armando De Arruda Pereira,1827,Ap 44,04309-011,Apartamento em condomínio,-46.640787,-23.642935,-46.640787,-23.642935
3503971,310119000100,310119000100,R Cotipora,0,NaN,04317-060,Terreno,-46.631329,-23.644451,-46.631329,-23.644451


In [45]:
df_enderecos['layer'] = 'address'

In [46]:
df_enderecos['source'] = 'IPTU2020'

In [47]:
from uuid import uuid4

In [48]:
df_enderecos['id'] = df_enderecos.index.to_series().map(lambda x: uuid4())

In [57]:
df_enderecos = df_enderecos[~df_enderecos.lat.isna()]

In [63]:
campos_export = ['id', 
                 'source', 
                 'layer', 
                 'lat', 
                 'lon', 
                 'name', 
                 'street', 
                 'number', 
                 'housenumber', 
                 'postcode']

In [64]:
df_enderecos[campos_export]

,id,source,layer,lat,lon,name,street,number,housenumber,postcode
0,4ff38326-412a-4151-84a5-111e2370b950,IPTU2020,address,-23.534357,-46.632863,001003000100,R S Caetano,13,NaN,01104-001
1,e70120c6-f410-4be3-8760-89fa53ddd3f8,IPTU2020,address,-23.534376,-46.632790,001003000200,R S Caetano,19,NaN,01104-001
2,16443ddb-150b-4231-8c89-67dff2695a99,IPTU2020,address,-23.534379,-46.632722,001003000300,R S Caetano,27,NaN,01104-001
3,8d75fdd0-f83e-4bf8-92b2-67111c5cee42,IPTU2020,address,-23.534385,-46.632657,001003000400,R S Caetano,33,NaN,01104-001
4,631d5fbb-2653-4217-88c8-74eb192ead14,IPTU2020,address,-23.534343,-46.632585,001003000500,R S Caetano,39,NaN,01104-001
...,...,...,...,...,...,...,...,...,...,...
3503967,66755183-e253-4ef0-8e33-298be61ec191,IPTU2020,address,-23.642935,-46.640787,310118023701,Av Eng Armando De Arruda Pereira,1827,Ap 41,04309-011
3503968,d94e4b20-3014-41e6-92bf-6d6e3e7a2bfa,IPTU2020,address,-23.642935,-46.640787,310118023801,Av Eng Armando De Arruda Pereira,1827,Ap 42,04309-011
3503969,dac8f7a9-18f5-409b-a6da-569dab093210,IPTU2020,address,-23.642935,-46.640787,310118023901,Av Eng Armando De Arruda Pereira,1827,Ap 43,04309-011
3503970,7debc3db-1ed2-439f-a69a-b15f9427e5a6,IPTU2020,address,-23.642935,-46.640787,310118024001,Av Eng Armando De Arruda Pereira,1827,Ap 44,04309-011


In [65]:
# compression_opts = dict(method='zip',
#                         archive_name='sao-paulo-address-IPTU-2021.csv',
#                        allowZip64=True) 

df_enderecos[campos_export].to_csv('resultados/enderecos-IPTU-2020-pelias.csv', 
            index=False)

In [61]:
# df_enderecos.loc[df_enderecos.X.notna(), campos_exportacao]

In [62]:
# df_iptu['NOME DE LOGRADOURO DO IMOVEL'].str.split(expand=True).stack().value_counts().head(100)